# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.6 MB/s eta 0:00:00


In [ ]:
!mkdir /content/dataset/
!mkdir -p /content/dataset/train2C/
!mkdir -p /content/dataset/val2C/
!tar -xvzf /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train.tgz -C /content/dataset/train2C/
!tar -xvzf /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val.tgz -C /content/dataset/val2C/
!mkdir -p /content/dataset/test2C/
!tar -xvzf /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/test/test2C.tgz -C /content/dataset/test2C/

Streaming output truncated to the last 5000 lines.
testX/xxx04139.jpg
testX/xxx05763.jpg
testX/xxx01199.jpg
testX/xxx03592.jpg
testX/xxx06086.jpg
testX/xxx02062.jpg
testX/xxx01124.jpg
testX/xxx00143.jpg
testX/xxx05437.jpg
testX/xxx00356.jpg
testX/xxx03594.jpg
testX/xxx02648.jpg
testX/xxx02681.jpg
testX/xxx01684.jpg
testX/xxx02143.jpg
testX/xxx01114.jpg
testX/xxx01592.jpg
testX/xxx02163.jpg
testX/xxx05441.jpg
testX/xxx02756.jpg
testX/xxx01785.jpg
testX/xxx00855.jpg
testX/xxx05754.jpg
testX/xxx00763.jpg
testX/xxx01156.jpg
testX/xxx03947.jpg
testX/xxx04798.jpg
testX/xxx05700.jpg
testX/xxx01011.jpg
testX/xxx05481.jpg
testX/xxx03439.jpg
testX/xxx02784.jpg
testX/xxx04103.jpg
testX/xxx04951.jpg
testX/xxx00865.jpg
testX/xxx05506.jpg
testX/xxx02922.jpg
testX/xxx02887.jpg
testX/xxx05907.jpg
testX/xxx02986.jpg
testX/xxx01465.jpg
testX/xxx03753.jpg
testX/xxx04834.jpg
testX/xxx03365.jpg
testX/xxx01115.jpg
testX/xxx02641.jpg
testX/xxx01640.jpg
testX/xxx00853.jpg
testX/xxx01432.jpg
testX/xxx05449.jpg

# Initialization

In [ ]:
%cd /content/drive/MyDrive/MaiaSpain/CAD/project/

/content/drive/.shortcut-targets-by-id/1fTsBspXCDEVY7q8PyaNkRq-51xfzOyIV/MaiaSpain/CAD/project


In [ ]:
import os
import numpy as np
from torch.utils.data import WeightedRandomSampler
from monai.transforms import (
    Compose, Rand2DElasticd, RandRotate90d, RandFlipd, RandAffined, ScaleIntensityd,
    RandCoarseShuffled, EnsureTyped, LoadImaged, Resized, ToTensord, NormalizeIntensityd
)
from monai.data import PersistentDataset, Dataset, DataLoader
from monai.losses import FocalLoss
from PIL import Image
import torch
from torch import nn, tensor
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, cohen_kappa_score
from tqdm import tqdm
from torchvision import models
from shutil import rmtree
from transformers import DeiTForImageClassification, DeiTConfig
import csv
from torch.nn import functional as F

# Dataloader

In [ ]:
cls_to_idx = {'nevus':0,'others':1}
idx_to_cls = {0:'nevus',1:'others'}

def get_weights(data_path):
  classes = os.listdir(data_path)
  num_samples = {cls: len(os.listdir(os.path.join(data_path, cls))) for cls in classes}
  weights = {cls: (1 / num_samples[cls]) / sum((1 / v for v in num_samples.values())) for cls in classes}
  return weights

def load_from_npz(file_name):
    data = np.load(file_name, allow_pickle=True)
    paths = data['paths']
    features = data['features']
    labels = data['labels']
    return paths, features, labels


def get_data(data_path, cls_weights, embeddings_path=None):
  data = []
  sample_weights = []
  classes = os.listdir(data_path)
  if embeddings_path is not None:
    npz_paths, npz_embeddings, npz_labels = load_from_npz(embeddings_path)
  for cls in classes:
      cls_path = os.path.join(data_path, cls)
      for image_name in os.listdir(cls_path):
          image_path = os.path.join(cls_path, image_name)
          if embeddings_path is not None:
            if image_path.replace('/content/dataset/','/Users/ayaelgebaly/Downloads/maiaUdg/CAD/DLproject/') not in npz_paths:
              print(image_path.replace('/content/dataset/','/Users/ayaelgebaly/Downloads/maiaUdg/CAD/DLproject/'))
              continue
            embedding = npz_embeddings[list(npz_paths).index(image_path.replace('/content/dataset/','/Users/ayaelgebaly/Downloads/maiaUdg/CAD/DLproject/'))]
            assert cls_to_idx[cls] == npz_labels[list(npz_paths).index(image_path.replace('/content/dataset/','/Users/ayaelgebaly/Downloads/maiaUdg/CAD/DLproject/'))]
            data.append({'image':image_path, 'label':cls_to_idx[cls], 'embedding':embedding})
          else:
            data.append({'image':image_path, 'label':cls_to_idx[cls]})
          sample_weights.append(cls_weights[cls])
  return data, sample_weights

In [ ]:
base_transforms = [
    LoadImaged('image',ensure_channel_first=True),
    Resized('image',(224, 224)),
    NormalizeIntensityd(['image'])
]

augmentations = [
    ScaleIntensityd('image',0,1),
    Rand2DElasticd('image',prob=0.75, spacing=56, magnitude_range=(1, 3), padding_mode="zeros"),
    RandRotate90d('image',prob=0.75, spatial_axes=[0, 1]),
    RandFlipd('image',prob=0.5, spatial_axis=0),
    RandFlipd('image',prob=0.5, spatial_axis=1),
    RandAffined('image',prob=0.75,rotate_range=(0.75, 0.75),padding_mode="zeros",),
    NormalizeIntensityd('image')
]

train_transform = Compose(base_transforms + augmentations)
val_transform = Compose(base_transforms)

# rmtree('/content/cache/',True)

cls_weights = get_weights("/content/dataset/train2C/train")
train_data, train_weights = get_data("/content/dataset/train2C/train", cls_weights, 'dataset/test/google_derm_train2c_embeddings.npz')
val_data, val_weights = get_data("/content/dataset/val2C/val", cls_weights)

train_dataset = PersistentDataset(data=train_data, transform=train_transform, cache_dir='/content/cache/train')
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))
train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, num_workers=os.cpu_count(), pin_memory=True)
val_dataset = PersistentDataset(data=val_data, transform=val_transform, cache_dir='/content/cache/val')
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=os.cpu_count(), pin_memory=True)

# Visualization

In [ ]:
# import matplotlib.pyplot as plt

# train_loader.dataset.transform.transforms = train_loader.dataset.transform.transforms+(ScaleIntensityd('image',0,1),)
# plt.figure(figsize=(18,10))
# batch = next(iter(train_loader))
# for i in range(32):
#   image, label = batch['image'][i], batch['label'][i]
#   plt.subplot(4,8,i+1)
#   plt.imshow(image.permute(1,2,0).numpy())
#   plt.title(idx_to_cls[label.item()])
#   plt.axis('off')
# plt.tight_layout()
# plt.show()

# Training and Evaluation

In [ ]:
def calculate_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    f1 = f1_score(labels, preds, average="macro")
    kappa = cohen_kappa_score(labels, preds)
    return accuracy, precision, recall, f1, kappa

def train_and_evaluate_model(model, train_loader, val_loader, device, epochs=30, min_delta=0.0001):
    model = model.to(device)
    projector = nn.Sequential(nn.Linear(768, 3072), nn.ReLU(), nn.Linear(3072, 6144), nn.ReLU()).to(device)
    cls_criterion = FocalLoss(to_onehot_y=True, use_softmax=True)
    distill_criterion = nn.KLDivLoss(reduction='batchmean')
    optimizer = torch.optim.AdamW(list(model.parameters()) + list(projector.parameters()), lr=1e-4, weight_decay=1e-4)
    scheduler=StepLR(optimizer, step_size=15, gamma=0.2)
    best_accuracy = 0
    save_dir = "models_alberb_2cls_distill/"
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(epochs):
        model.train()
        epoch_cls_loss, epoch_distill_loss = 0, 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            images, labels, gt_embedding = batch['image'].to(device), batch['label'].to(device), batch['embedding'].to(device)
            optimizer.zero_grad()
            outputs = model(images)
            outputs, embedding = outputs.logits, outputs.hidden_states[-1][:, -1, :]
            cls_loss = cls_criterion(outputs, labels)
            distill_loss = distill_criterion(F.log_softmax(projector(embedding), dim=-1), F.softmax(gt_embedding, dim=-1))
            loss = cls_loss + distill_loss
            loss.backward()
            optimizer.step()
            epoch_cls_loss += cls_loss.item()
            epoch_distill_loss += distill_loss.item()
        scheduler.step()

        print(f"Epoch {epoch + 1}/{epochs}, Classification Loss: {epoch_cls_loss / len(train_loader):.6f}, Distill Loss: {epoch_distill_loss / len(train_loader):.6f}")

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch['image'].to(device), batch['label'].to(device)
                outputs = model(images)
                outputs = outputs.logits
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy, precision, recall, f1, kappa = calculate_metrics(all_labels, all_preds)
        cm = confusion_matrix(all_labels, all_preds)
        print(f"Validation Metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Kappa: {kappa:.4f}")
        print(f"Confusion Matrix:\n{cm}")

        if accuracy > best_accuracy + min_delta:
            model_path = os.path.join(save_dir, f"best_model_{accuracy:.4f}.pth")
            torch.save(model.state_dict(), model_path)
            if os.path.exists(os.path.join(save_dir, f"best_model_{best_accuracy:.4f}.pth")):
              os.remove(os.path.join(save_dir, f"best_model_{best_accuracy:.4f}.pth"))
            best_accuracy = accuracy
            print(f"Model saved to {model_path}")

In [ ]:
from transformers import DeiTForImageClassification, DeiTConfig

config = DeiTConfig.from_pretrained("facebook/deit-base-distilled-patch16-224")
config.num_labels, config.output_hidden_states = 2, True
model = DeiTForImageClassification.from_pretrained("facebook/deit-base-distilled-patch16-224", config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Experiments

In [ ]:
train_and_evaluate_model(
    model, train_loader, val_loader, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=100
)

Epoch 1/100:   0%|          | 0/475 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return t

Epoch 1/100, Classification Loss: 0.053860, Distill Loss: 0.193884



/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-

Validation Metrics - Accuracy: 0.8388, Precision: 0.8422, Recall: 0.8396, F1: 0.8386, Kappa: 0.6780
Confusion Matrix:
[[1525  406]
 [ 206 1659]]
Model saved to models_alberb_2cls_distill/best_model_0.8388.pth


Epoch 2/100:   0%|          | 0/475 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return t

...

Epoch 22/100:   0%|          | 0/475 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

Epoch 22/100, Classification Loss: 0.000543, Distill Loss: 0.036437



/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-

Validation Metrics - Accuracy: 0.9170, Precision: 0.9172, Recall: 0.9169, F1: 0.9170, Kappa: 0.8339
Confusion Matrix:
[[1787  144]
 [ 171 1694]]


Epoch 23/100:   0%|          | 0/475 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

# TTA

In [ ]:
tta_augs = Compose(augmentations)
def augment(images):
  images = torch.stack([tta_augs({'image':image})['image'] for image in images], 0)
  return images

In [ ]:
tta_iters = 7
model.to('cuda')
model.load_state_dict(torch.load('models_alberb_2cls_distill/best_model_0.9181.pth'))
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        images, labels = batch['image'].to('cuda'), batch['label'].to('cuda')
        outputs = torch.softmax(model(images).logits,1)
        for _ in range(tta_iters):
          outputs += torch.softmax(model(augment(images.clone())).logits,1)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    accuracy, precision, recall, f1, kappa = calculate_metrics(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)
    print(f"Validation Metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Kappa: {kappa:.4f}")
    print(f"Confusion Matrix:\n{cm}")

<ipython-input-11-294a00ce8f65>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models_alberb_2cls_distill/best_model_0.9181.pth'))


Validation Metrics - Accuracy: 0.9207, Precision: 0.9215, Recall: 0.9204, F1: 0.9206, Kappa: 0.8413
Confusion Matrix:
[[1815  116]
 [ 185 1680]]


# Prediction

In [ ]:
def get_test_data(data_path):
  data = []
  for image_name in sorted(os.listdir(data_path)):
      image_path = os.path.join(data_path, image_name)
      data.append({'image':image_path})
  return data
test_transforms = Compose(base_transforms)
test_data = get_test_data("/content/dataset/test2C/testX")
test_dataset = Dataset(data=test_data, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=os.cpu_count(), pin_memory=True)

In [ ]:
model_path = 'models_alberb_2cls_distill/best_model_0.9181.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval();
model = model.to(device)

<ipython-input-9-d870b3c99129>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [ ]:
tta_augs = Compose(augmentations)
def augment(images):
  images = torch.stack([tta_augs({'image':image})['image'] for image in images], 0)
  return images

tta_iters = 7
all_preds = []
with torch.no_grad():
    for batch in test_loader:
        images = batch['image'].to(device)
        outputs = torch.softmax(model(images).logits,1)
        for _ in range(tta_iters):
          outputs += torch.softmax(model(augment(images.clone())).logits,1)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
with open(model_path.replace('.pth','_test_results.csv'), "w", newline="") as f:
  csv.writer(f).writerows([[x] for x in all_preds])

In [ ]:
print(np.unique(all_preds, return_counts=True))

(array([0, 1]), array([3365, 2975]))
